In [1]:
import os
import json
import numpy as np

# Strategy:
# Extract pos_ner + embeddings & rolling embeddings
# Flag bad ner, use POS to get the numbers and potential other bad stuff
# Project embeddings & rolling embeddings, cosine sim with all other clusters ?
# Then rules for numbers

text_path = '../example_conversations/english/labelled_text_conv_1.json'

with open(text_path, 'r') as f:
    labelled_data = json.load(f)

unlabelled_data = [list(i.keys())[0] for i in labelled_data]


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_ID = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
MODEL_ID = 'google/gemma-3-1b-it'

MODEL_ID = 'google/gemma-3-4b-it-qat-q4_0-gguf'

MODEL_ID = 'vinimuchulski/gemma-3-4b-it-qat-q4_0-gguf'
MODEL_ID = 'google/gemma-3-4b-it'
# MODEL_ID = 'meta-llama/Llama-3.2-1B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,
                                             torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                                             device_map='auto')

def call_llm(prompt: str, max_tokens=10):

    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))



Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.39s/it]
Some parameters are on the meta device because they were offloaded to the disk.


In [16]:
call_llm('Hello, who are you ?')

RuntimeError: layer 27 has not been mapped to a device.

In [59]:
def call_llm(prompt: str, max_tokens=10):

    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [64]:
with open('../prompt.txt', 'r') as f:
    prompt = f.read()

rolled_window_data = []
win_down, win_up = -5, 5
for idx, word in enumerate(unlabelled_data):
    start, end = idx + win_down, idx + win_up
    start = max(0, start)
    end = min(end, len(unlabelled_data))

    rolled_window_data.append((' '.join(unlabelled_data[start:end]), word, idx))

rolled_window_data

[('Hi can you believe the', 'Hi', 0),
 ('Hi can you believe the bus', 'can', 1),
 ('Hi can you believe the bus was', 'you', 2),
 ('Hi can you believe the bus was late', 'believe', 3),
 ('Hi can you believe the bus was late again', 'the', 4),
 ('Hi can you believe the bus was late again this', 'bus', 5),
 ('can you believe the bus was late again this morning', 'was', 6),
 ('you believe the bus was late again this morning Yeah', 'late', 7),
 ('believe the bus was late again this morning Yeah it’s', 'again', 8),
 ('the bus was late again this morning Yeah it’s always', 'this', 9),
 ('bus was late again this morning Yeah it’s always Line', 'morning', 10),
 ('was late again this morning Yeah it’s always Line 25', 'Yeah', 11),
 ('late again this morning Yeah it’s always Line 25 I', 'it’s', 12),
 ('again this morning Yeah it’s always Line 25 I was', 'always', 13),
 ('this morning Yeah it’s always Line 25 I was supposed', 'Line', 14),
 ('morning Yeah it’s always Line 25 I was supposed to', '25

In [67]:
def inject_prompt(sentence, word):

    return prompt.replace('$sentence$', sentence).replace('$word$', word)

sentence, word = rolled_window_data[378][:2]

injected_prompt = inject_prompt(sentence.lower(), word.lower())

print(injected_prompt)

Task: You are a de-identification model. Your job is to classify a target word from a sentence as either safe to publish (0) or sensitive personal information (1) according to the HIPAA Privacy Rule.

The HIPAA rule defines 18 types of sensitive information. If the word is part of any of these identifiers, label it as 1 (sensitive). Otherwise, label it as 0 (safe).

HIPAA identifiers include:
- Names
- Addresses smaller than a state (e.g. street, city, ZIP)
- Dates (except year)
- Phone/fax numbers
- Email addresses
- Social Security numbers
- Medical record / account / license numbers
- Vehicle / device IDs
- URLs / IP addresses
- Fingerprints / voiceprints
- Photos / any other unique identifying detail

Answer with just one digit:
- 1 = sensitive
- 0 = safe

Examples:

Sentence: My name is John Smith and I live in Geneva.
Word: John
Answer: 1

Sentence: My name is John Smith and I live in Geneva.
Word: Geneva
Answer: 1

Sentence: My name is John Smith and I live in Geneva.
Word: live

In [68]:
response = call_llm(injected_prompt, 2)

response[-5:]

'er: 0'

In [74]:
labelled_data = []

for sentence, word, idx in rolled_window_data[:75]:
    injected_prompt = inject_prompt(sentence, word)

    response = call_llm(injected_prompt, 2)[-3:]
    labelled_data.append((word, response))

    print(idx+1, '/75')

for w, l in labelled_data:

    code = 32 if l[-1] == '0' else 31
    print(f'\033[{code}m {w} \033[0m', end='')

1 /75
2 /75
3 /75
4 /75
5 /75
6 /75
7 /75
8 /75
9 /75
10 /75
11 /75
12 /75
13 /75
14 /75
15 /75
16 /75
17 /75
18 /75
19 /75
20 /75
21 /75
22 /75
23 /75
24 /75
25 /75
26 /75
27 /75
28 /75
29 /75
30 /75
31 /75
32 /75
33 /75
34 /75
35 /75
36 /75
37 /75
38 /75
39 /75
40 /75
41 /75
42 /75
43 /75
44 /75
45 /75
46 /75
47 /75
48 /75
49 /75
50 /75
51 /75
52 /75
53 /75
54 /75
55 /75
56 /75
57 /75
58 /75
59 /75
60 /75
61 /75
62 /75
63 /75
64 /75
65 /75
66 /75
67 /75
68 /75
69 /75
70 /75
71 /75
72 /75
73 /75
74 /75
75 /75
 Hi  can  you  believe  the  bus  was  late  again  this  morning  Yeah  it’s  always  Line  25  I  was  supposed  to  be  at  4  Place  du  Marché  by  8h30  Did  you  see  that  new  café  on  Rue  de  la  Paix  No  but  Clara  mentioned  it  to  me  Speaking  of  Clara’s  birthday  is  on  March  18th  right  Yes  but  she’s  having  the  party  next  Friday  at  her  place  in  Lausanne  I’ll  need  to  check  with  my  wife 

 Hi  can  you  believe  the  bus  was  late  again  this  morning  Yeah  it’s  always  Line  25  I  was  supposed  to  be  at  4  Place  du  Marché  by  8h30  Did  you  see  that  new  café  on  Rue  de  la  Paix  No  but  Clara  mentioned  it  to  me  Speaking  of  Clara’s  birthday  is  on  March  18th  right  Yes  but  she’s  having  the  party  next  Friday  at  her  place  in  Lausanne  I’ll  need  to  check  with  my  wife 